### Desde https://causalnex.readthedocs.io/

# Tutorial CausalNex 


Este tutorial es un ejemplo de uso de CausalNex para estimar si un estudiante aprobabará o suspenderá un examen, basándose en características como relaciones familiares, soporte escolar y otros. Se usará el dataset de [Student Performance](https://archive.ics.uci.edu/ml/datasets/Student+Performance) publicado en [UCI Machine Learning Repository.](http://archive.ics.uci.edu/ml/index.php)

## Structure Learning

Defining the structure of a Bayesian Network (BN) model can be done based on machine learning, domain knowledge, or a combination of both, where experts and algorithms contribute as equal partners.

Regardless of the approach, it is important to validate the structure by evaluating the BN - this will be covered later in the tutorial. In this section, we will focus on how to define a structure.



## Structure from Domain Knowledge

We can manually define a structure model by specifying the relationships between different features.

First, we must create an empty structure model.

In [1]:
from causalnex.structure import StructureModel

sm=StructureModel()
sm